In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [20]:
from src.inference import get_feature_store

In [21]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-02-19 20:56:37,824 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-19 20:56:38,702 INFO: Initializing external client
2025-02-19 20:56:38,703 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-19 20:56:40,512 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213633
Fetching data from 2025-01-22 01:56:37.741190+00:00 to 2025-02-20 00:56:37.741190+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (6.34s) 


In [22]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-01-22 02:00:00+00:00,2,0
1,2025-01-22 03:00:00+00:00,2,0
2,2025-01-22 04:00:00+00:00,2,0
3,2025-01-22 05:00:00+00:00,2,0
4,2025-01-22 06:00:00+00:00,2,0
...,...,...,...
175830,2025-02-19 20:00:00+00:00,263,110
175831,2025-02-19 21:00:00+00:00,263,107
175832,2025-02-19 22:00:00+00:00,263,72
175833,2025-02-19 23:00:00+00:00,263,31


In [23]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 175835 entries, 0 to 181905
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         175835 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  175835 non-null  int32                  
 2   rides               175835 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.0 MB


In [24]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [25]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 175835 entries, 0 to 181905
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         175835 non-null  datetime64[us]
 1   pickup_location_id  175835 non-null  int32         
 2   rides               175835 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.0 MB


In [26]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [27]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,67,2025-02-02 11:00:00
1,0,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,1,147,2025-02-17 12:00:00
2,3,2,2,2,1,4,2,2,2,1,...,0,3,2,5,2,3,8,3,65,2025-02-12 15:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,156,2025-02-05 03:00:00
4,1,0,0,0,0,0,1,2,0,0,...,1,0,0,0,0,0,0,1,155,2025-02-01 21:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,0,0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,207,2025-01-29 15:00:00
249,0,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,119,2025-02-13 10:00:00
250,2,0,1,0,2,0,0,1,0,4,...,0,0,1,0,7,3,6,0,12,2025-02-15 20:00:00
251,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,31,2025-02-07 20:00:00


In [28]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-02-19 20:56:53,627 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-19 20:56:53,877 INFO: Initializing external client
2025-02-19 20:56:53,878 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-19 20:56:55,262 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213633
Fetching data from 2025-01-22 01:56:53.627527+00:00 to 2025-02-20 00:56:53.627527+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.36s) 


In [29]:
current_date

Timestamp('2025-02-20 01:56:53.627527+0000', tz='Etc/UTC')

In [30]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-02-19 02:00:00
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,3,2025-02-19 02:00:00
2,0,0,0,0,0,6,3,8,1,2,...,3,2,5,2,2,0,0,2,4,2025-02-19 02:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-02-19 02:00:00
4,1,0,0,1,0,2,3,4,6,0,...,0,2,1,2,1,2,1,0,7,2025-02-19 02:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,0,0,0,0,1,0,0,1,0,0,...,0,1,0,0,0,0,0,0,259,2025-02-19 02:00:00
249,0,0,0,1,2,1,1,3,0,2,...,0,0,0,0,0,0,0,0,260,2025-02-19 02:00:00
250,4,0,0,0,1,9,16,14,18,20,...,46,32,23,17,20,3,3,0,261,2025-02-19 02:00:00
251,4,3,2,12,60,133,133,159,106,96,...,69,40,39,30,11,6,4,1,262,2025-02-19 02:00:00


In [31]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-02-19 20:57:04,507 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-19 20:57:04,519 INFO: Initializing external client
2025-02-19 20:57:04,520 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-19 20:57:05,589 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213633

In [32]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [33]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,1.0
3,6,0.0
4,7,0.0
...,...,...
248,259,0.0
249,260,0.0
250,261,1.0
251,262,1.0


In [34]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([132,  48, 230, 249,  79, 114, 100, 164, 186, 231])